In [ ]:
#자체 데이터 제작에 쓰인 코드
#동영상 촬영 후 해당 동영상을 YOLO를 적용하여 bounding box 라벨 정보를 생성하고, 이를 검토하여 학습데이터로 활용
#라벨값은 다른 코드에서 수정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Bigproject'

In [ ]:
## colab에서 세션 재시작을 요구하는 팝업이 뜨면 재시작 누르세요.
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 20.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import zipfile

In [ ]:
import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os
from tqdm import tqdm

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os
from tqdm import tqdm

# 모델 경로 설정 및 불러오기
model_path = '/content/drive/MyDrive/Bigproject/model/m/yolom_imgsz=750_a_epoch_best.pt'
model = YOLO(model_path)

# 클래스 이름 설정
class_names = [
    "bread", "snack", "coffee", "juice", "noodle", "seasoning", "shampoo",
    "soap", "bodywash", "moisturizer", "detergent", "toothpaste", "tata_salt",
    "cheese", "egg", "milk", "meat", "sausages", "beverage", "canned_food",
    "miscellaneous_item", "apple", "banana", "tomato", "cucumber", "carrot"
]

# 동영상 폴더 경로 설정
video_folder_path = '/content/drive/MyDrive/Bigproject/Video/train1'
video_files = [f for f in os.listdir(video_folder_path) if f.endswith(('.mp4', '.avi', '.mkv'))]  # 지원 파일 형식

if not video_files:
    print("동영상 파일이 없습니다.")
    exit()

print(f"🎥 총 {len(video_files)}개의 동영상을 처리합니다.")

# 이미지 확인 플래그 (True로 설정 시 감지된 이미지를 출력)
show_images = True

for video_file in video_files:
    data_name = os.path.splitext(video_file)[0]  # 동영상 파일 이름에서 확장자 제거
    video_path = os.path.join(video_folder_path, video_file)

    # 저장 폴더 설정
    base_save_path = f'/content/drive/MyDrive/Bigproject/data_make/{data_name}'
    label_save_path = f"{base_save_path}/label"
    image_save_path = f"{base_save_path}/image"
    imagecheck_save_path = f"{base_save_path}/imagecheck"

    # 폴더 생성
    os.makedirs(label_save_path, exist_ok=True)
    os.makedirs(image_save_path, exist_ok=True)
    os.makedirs(imagecheck_save_path, exist_ok=True)

    # 동영상 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 동영상을 열 수 없습니다: {video_file}")
        continue

    # 동영상 총 프레임 수 계산
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎥 [{video_file}] 총 프레임 수: {total_frames}")

    frame_count = 0

    # 진행률 표시를 위한 tqdm 설정
    progress_bar = tqdm(total=total_frames, desc=f"🔄 {data_name} 진행률")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 모델을 통해 객체 감지
        results = model(frame, conf=0.5, iou=0.45)

        # 어노테이션 저장용 리스트
        annotations = []

        # 감지된 객체가 포함된 이미지를 위한 복사본
        frame_with_boxes = frame.copy()

        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
                cls_id = int(box.cls[0])  # 클래스 ID
                conf = box.conf[0]  # 신뢰도 점수

                # 이미지 크기 가져오기
                img_height, img_width = frame.shape[:2]

                # 정규화된 좌표 계산 (YOLO 형식)
                x_center = ((x1 + x2) / 2) / img_width
                y_center = ((y1 + y2) / 2) / img_height
                width = (x2 - x1) / img_width
                height = (y2 - y1) / img_height

                # 어노테이션 형식: class_id x_center y_center width height
                annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

                # 바운딩 박스 및 텍스트 추가
                label = f"{class_names[cls_id]} ({conf:.2f})"
                color = (0, 255, 0)  # 초록색
                cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame_with_boxes, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # 텍스트 파일로 어노테이션 저장
        if annotations:
            label_file_path = os.path.join(label_save_path, f"{data_name}_frame_{frame_count:04d}.txt")  # 라벨 파일
            with open(label_file_path, 'w') as f:
                f.write('\n'.join(annotations))

            # 라벨과 같은 이름으로 원본 이미지 저장
            image_file_path = os.path.join(image_save_path, f"{data_name}_frame_{frame_count:04d}.jpg")
            cv2.imwrite(image_file_path, frame)

            # 감지된 객체를 표시한 이미지 저장
            imagecheck_file_path = os.path.join(imagecheck_save_path, f"{data_name}_frame_{frame_count:04d}_check.jpg")
            cv2.imwrite(imagecheck_file_path, frame_with_boxes)

            # 주기적으로 이미지 확인
            if show_images and frame_count % 10 == 0:  # 10 프레임마다 출력
                print(f"📸 Frame {frame_count}: 객체 표시 이미지")
                cv2_imshow(frame_with_boxes)

        # tqdm 진행률 업데이트
        progress_bar.update(1)

        frame_count += 1

    cap.release()
    progress_bar.close()
    print(f"📂 [{data_name}] 모든 데이터가 {base_save_path}에 저장되었습니다.")

print("✅ 모든 동영상 처리가 완료되었습니다.")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
ㅁㅁ

NameError: name 'ᄆᄆ' is not defined

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os
from tqdm import tqdm

# 모델 경로 설정 및 불러오기
model_path = '/content/drive/MyDrive/Bigproject/model/m/yolom_imgsz=750_a_epoch_best.pt'
model = YOLO(model_path)

# 클래스 이름 설정
class_names = [
    "bread", "snack", "coffee", "juice", "noodle", "seasoning", "shampoo",
    "soap", "bodywash", "moisturizer", "detergent", "toothpaste", "tata_salt",
    "cheese", "egg", "milk", "meat", "sausages", "beverage", "canned_food",
    "miscellaneous_item", "apple", "banana", "tomato", "cucumber", "carrot"
]

# 저장 폴더 및 데이터 이름 설정
data_name = 'poka'  # 데이터 이름
base_save_path = f'/content/drive/MyDrive/Bigproject/data_make/{data_name}'
label_save_path = f"{base_save_path}/label"
image_save_path = f"{base_save_path}/image"
imagecheck_save_path = f"{base_save_path}/imagecheck"

# 폴더 생성
os.makedirs(label_save_path, exist_ok=True)
os.makedirs(image_save_path, exist_ok=True)
os.makedirs(imagecheck_save_path, exist_ok=True)

# 동영상 파일 경로 설정
video_path = '/content/drive/MyDrive/Bigproject/Video/20250128_154552.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("동영상을 열 수 없습니다.")
    exit()

# 동영상 총 프레임 수 계산
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎥 동영상 총 프레임 수: {total_frames}")

frame_count = 0

# 진행률 표시를 위한 tqdm 설정
progress_bar = tqdm(total=total_frames, desc="🔄 진행률")

# 이미지 확인 플래그 (True로 설정 시 감지된 이미지를 출력)
show_images = True

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델을 통해 객체 감지
    results = model(frame, conf=0.5, iou=0.45)

    # 어노테이션 저장용 리스트
    annotations = []

    # 감지된 객체가 포함된 이미지를 위한 복사본
    frame_with_boxes = frame.copy()

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            cls_id = int(box.cls[0])  # 클래스 ID
            conf = box.conf[0]  # 신뢰도 점수

            # 이미지 크기 가져오기
            img_height, img_width = frame.shape[:2]

            # 정규화된 좌표 계산 (YOLO 형식)
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            # 어노테이션 형식: class_id x_center y_center width height
            annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # 바운딩 박스 및 텍스트 추가
            label = f"{class_names[cls_id]} ({conf:.2f})"
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame_with_boxes, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 텍스트 파일로 어노테이션 저장
    if annotations:
        label_file_path = os.path.join(label_save_path, f"{data_name}_frame_{frame_count:04d}.txt")  # 라벨 파일
        with open(label_file_path, 'w') as f:
            f.write('\n'.join(annotations))

        # 라벨과 같은 이름으로 원본 이미지 저장
        image_file_path = os.path.join(image_save_path, f"{data_name}_frame_{frame_count:04d}.jpg")
        cv2.imwrite(image_file_path, frame)

        # 감지된 객체를 표시한 이미지 저장
        imagecheck_file_path = os.path.join(imagecheck_save_path, f"{data_name}_frame_{frame_count:04d}_check.jpg")
        cv2.imwrite(imagecheck_file_path, frame_with_boxes)

        # 주기적으로 이미지 확인
        if show_images and frame_count % 10 == 0:  # 10 프레임마다 출력
            print(f"📸 Frame {frame_count}: 객체 표시 이미지")
            cv2_imshow(frame_with_boxes)

    # tqdm 진행률 업데이트
    progress_bar.update(1)

    frame_count += 1

cap.release()
progress_bar.close()
print(f"📂 모든 데이터가 {base_save_path}에 저장되었습니다.")


In [ ]:
ㅁㅁ

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os

# 모델 경로 설정 및 불러오기
model_path = '/content/drive/MyDrive/Bigproject/model/m/yolom_imgsz=750_a_epoch_best.pt'
model = YOLO(model_path)

# 클래스 이름 설정
class_names = [
    "bread", "snack", "coffee", "juice", "noodle", "seasoning", "shampoo",
    "soap", "bodywash", "moisturizer", "detergent", "toothpaste", "tata_salt",
    "cheese", "egg", "milk", "meat", "sausages", "beverage", "canned_food",
    "miscellaneous_item", "apple", "banana", "tomato", "cucumber", "carrot"
]

# 저장 폴더 및 데이터 이름 설정
data_name = 'poka'  # 데이터 이름
base_save_path = f'/content/drive/MyDrive/Bigproject/data_make/{data_name}'
label_save_path = f"{base_save_path}/label"
image_save_path = f"{base_save_path}/image"
imagecheck_save_path = f"{base_save_path}/imagecheck"

# 폴더 생성
os.makedirs(label_save_path, exist_ok=True)
os.makedirs(image_save_path, exist_ok=True)
os.makedirs(imagecheck_save_path, exist_ok=True)

# 동영상 파일 경로 설정
#video_path = '/content/drive/MyDrive/Bigproject/Video/20250128_150638.mp4'
video_path = '/content/drive/MyDrive/Bigproject/Video/20250128_154552.mp4'

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("동영상을 열 수 없습니다.")
    exit()

# 동영상 총 프레임 수 계산
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎥 동영상 총 프레임 수: {total_frames}")

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델을 통해 객체 감지
    results = model(frame, conf=0.5, iou=0.45)

    # 어노테이션 저장용 리스트
    annotations = []

    # 감지된 객체가 포함된 이미지를 위한 복사본
    frame_with_boxes = frame.copy()

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            cls_id = int(box.cls[0])  # 클래스 ID
            conf = box.conf[0]  # 신뢰도 점수

            # 이미지 크기 가져오기
            img_height, img_width = frame.shape[:2]

            # 정규화된 좌표 계산 (YOLO 형식)
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            # 어노테이션 형식: class_id x_center y_center width height
            annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # 바운딩 박스 및 텍스트 추가
            label = f"{class_names[cls_id]} ({conf:.2f})"
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame_with_boxes, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 텍스트 파일로 어노테이션 저장
    if annotations:
        label_file_path = os.path.join(label_save_path, f"{data_name}_frame_{frame_count:04d}.txt")  # 라벨 파일
        with open(label_file_path, 'w') as f:
            f.write('\n'.join(annotations))

        # 라벨과 같은 이름으로 원본 이미지 저장
        image_file_path = os.path.join(image_save_path, f"{data_name}_frame_{frame_count:04d}.jpg")
        cv2.imwrite(image_file_path, frame)

        # 감지된 객체를 표시한 이미지 저장
        imagecheck_file_path = os.path.join(imagecheck_save_path, f"{data_name}_frame_{frame_count:04d}_check.jpg")
        cv2.imwrite(imagecheck_file_path, frame_with_boxes)

        # **Colab 출력 창에 감지된 객체 표시 이미지 출력**
        print(f"📸 Frame {frame_count}: 객체 표시 이미지")
        cv2_imshow(frame_with_boxes)

    # 진행 상황 출력
    progress = (frame_count + 1) / total_frames * 100
    print(f"🔄 진행률: {progress:.2f}% | Frame: {frame_count}/{total_frames} | 객체 감지: {len(annotations)}개")

    frame_count += 1

cap.release()
print(f"📂 모든 데이터가 {base_save_path}에 저장되었습니다.")


In [ ]:
ㅁㅁ

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os

# 모델 경로 설정 및 불러오기
model_path = '/content/drive/MyDrive/Bigproject/model/m/yolom_imgsz=750_a_epoch_best.pt'
model = YOLO(model_path)

# 클래스 이름 설정
class_names = [
    "bread", "snack", "coffee", "juice", "noodle", "seasoning", "shampoo",
    "soap", "bodywash", "moisturizer", "detergent", "toothpaste", "tata_salt",
    "cheese", "egg", "milk", "meat", "sausages", "beverage", "canned_food",
    "miscellaneous_item", "apple", "banana", "tomato", "cucumber", "carrot"
]

# 저장 폴더 및 데이터 이름 설정
data_name = 'cookie'
base_save_path = f'/content/drive/MyDrive/Bigproject/data_make/{data_name}'
label_save_path = f"{base_save_path}/label"
image_save_path = f"{base_save_path}/image"
imagecheck_save_path = f"{base_save_path}/imagecheck"

# 폴더 생성
os.makedirs(label_save_path, exist_ok=True)
os.makedirs(image_save_path, exist_ok=True)
os.makedirs(imagecheck_save_path, exist_ok=True)

# 동영상 파일 경로 설정
video_path = '/content/drive/MyDrive/Bigproject/Video/20250128_150638.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("동영상을 열 수 없습니다.")
    exit()

# 동영상 총 프레임 수 계산
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎥 동영상 총 프레임 수: {total_frames}")

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델을 통해 객체 감지
    results = model(frame, conf=0.5, iou=0.45)

    # 어노테이션 저장용 리스트
    annotations = []

    # 감지된 객체가 포함된 이미지를 위한 복사본
    frame_with_boxes = frame.copy()

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            cls_id = int(box.cls[0])  # 클래스 ID
            conf = box.conf[0]  # 신뢰도 점수

            # 이미지 크기 가져오기
            img_height, img_width = frame.shape[:2]

            # 정규화된 좌표 계산 (YOLO 형식)
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            # 어노테이션 형식: class_id x_center y_center width height
            annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # 바운딩 박스 및 텍스트 추가
            label = f"{class_names[cls_id]} ({conf:.2f})"
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame_with_boxes, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 텍스트 파일로 어노테이션 저장
    if annotations:
        label_file_path = os.path.join(label_save_path, f"frame_{frame_count:04d}.txt")  # 라벨 파일
        with open(label_file_path, 'w') as f:
            f.write('\n'.join(annotations))

        # 라벨과 같은 이름으로 원본 이미지 저장
        image_file_path = os.path.join(image_save_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(image_file_path, frame)

        # 감지된 객체를 표시한 이미지 저장
        imagecheck_file_path = os.path.join(imagecheck_save_path, f"frame_{frame_count:04d}_check.jpg")
        cv2.imwrite(imagecheck_file_path, frame_with_boxes)

        # **Colab 출력 창에 감지된 객체 표시 이미지 출력**
        print(f"📸 Frame {frame_count}: 객체 표시 이미지")
        cv2_imshow(frame_with_boxes)

    # 진행 상황 출력
    progress = (frame_count + 1) / total_frames * 100
    print(f"🔄 진행률: {progress:.2f}% | Frame: {frame_count}/{total_frames} | 객체 감지: {len(annotations)}개")

    frame_count += 1

cap.release()
print(f"📂 모든 데이터가 {base_save_path}에 저장되었습니다.")


In [ ]:
ㅁㅁ

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os

# 모델 경로 설정 및 불러오기
model_path = '/content/drive/MyDrive/Bigproject/model/m/yolom_imgsz=750_a_epoch_best.pt'
model = YOLO(model_path)

# 클래스 이름 설정
class_names = [
    "bread", "snack", "coffee", "juice", "noodle", "seasoning", "shampoo",
    "soap", "bodywash", "moisturizer", "detergent", "toothpaste", "tata_salt",
    "cheese", "egg", "milk", "meat", "sausages", "beverage", "canned_food",
    "miscellaneous_item", "apple", "banana", "tomato", "cucumber", "carrot"
]

# 저장 폴더 및 데이터 이름 설정
data_name = 'cookie'
base_save_path = f'/content/drive/MyDrive/Bigproject/data_make/{data_name}'
label_save_path = f"{base_save_path}/label"
image_save_path = f"{base_save_path}/image"
imagecheck_save_path = f"{base_save_path}/imagecheck"

# 폴더 생성
os.makedirs(label_save_path, exist_ok=True)
os.makedirs(image_save_path, exist_ok=True)
os.makedirs(imagecheck_save_path, exist_ok=True)

# 동영상 파일 경로 설정
video_path = '/content/drive/MyDrive/Bigproject/Video/20250128_150638.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("동영상을 열 수 없습니다.")
    exit()

# 동영상 총 프레임 수 계산
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎥 동영상 총 프레임 수: {total_frames}")

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델을 통해 객체 감지
    results = model(frame, conf=0.5, iou=0.45)

    # 어노테이션 저장용 리스트
    annotations = []

    # 감지된 객체가 포함된 이미지를 위한 복사본
    frame_with_boxes = frame.copy()

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            cls_id = int(box.cls[0])  # 클래스 ID
            conf = box.conf[0]  # 신뢰도 점수

            # 이미지 크기 가져오기
            img_height, img_width = frame.shape[:2]

            # 정규화된 좌표 계산 (YOLO 형식)
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            # 어노테이션 형식: class_id x_center y_center width height
            annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # 바운딩 박스 및 텍스트 추가
            label = f"{class_names[cls_id]} ({conf:.2f})"
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame_with_boxes, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 텍스트 파일로 어노테이션 저장
    if annotations:
        label_file_path = os.path.join(label_save_path, f"frame_{frame_count:04d}.txt")  # 라벨 파일
        with open(label_file_path, 'w') as f:
            f.write('\n'.join(annotations))

        # 라벨과 같은 이름으로 원본 이미지 저장
        image_file_path = os.path.join(image_save_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(image_file_path, frame)

        # 감지된 객체를 표시한 이미지 저장
        imagecheck_file_path = os.path.join(imagecheck_save_path, f"frame_{frame_count:04d}_check.jpg")
        cv2.imwrite(imagecheck_file_path, frame_with_boxes)

    # 진행 상황 출력
    progress = (frame_count + 1) / total_frames * 100
    print(f"🔄 진행률: {progress:.2f}% | Frame: {frame_count}/{total_frames} | 객체 감지: {len(annotations)}개")

    frame_count += 1

cap.release()
print(f"📂 모든 데이터가 {base_save_path}에 저장되었습니다.")
